In [2]:
# get the tensorflow version of samplernn
!git clone https://github.com/multimedia-eurecat/SampleRNN 

fatal: destination path 'SampleRNN' already exists and is not an empty directory.


In [0]:
# add samplernn to path
import sys
import os
sys.path.insert(1, 'SampleRNN')

In [4]:
# update if necessary
os.chdir("SampleRNN")
!git pull
os.chdir("..")

Already up to date.


In [5]:
# install librosa if needed
!pip install librosa #probably already satisfied ... 

In [0]:
# create logdir folder
!mkdir logdir

In [0]:
# create output folder for generated audio
!mkdir generated

In [8]:
# create corpus folder, add some data
!mkdir corpus
os.chdir("corpus")
# Trying to train SampleRNN on soundscapes, so here's a dataset that 
# has different types and mixes of soundscapes ... nature, industry etc ...
!curl https://vault.sfu.ca/index.php/s/HztBdHn5GVMLFA7/download --output Emo-Soundscapes.zip
!unzip Emo-Soundscapes.zip
os.chdir("..")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  518M  100  518M    0     0  2466k      0  0:03:35  0:03:35 --:--:-- 2521k
Archive:  Emo-Soundscapes.zip
   creating: Emo-Soundscapes/
  inflating: Emo-Soundscapes/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/Emo-Soundscapes/
  inflating: __MACOSX/Emo-Soundscapes/._.DS_Store  
   creating: Emo-Soundscapes/Emo-Soundscapes-Audio/
  inflating: Emo-Soundscapes/Emo-Soundscapes-Audio/.DS_Store  
   creating: __MACOSX/Emo-Soundscapes/Emo-Soundscapes-Audio/
  inflating: __MACOSX/Emo-Soundscapes/Emo-Soundscapes-Audio/._.DS_Store  
   creating: Emo-Soundscapes/Emo-Soundscapes-Audio/600_Sounds/
   creating: Emo-Soundscapes/Emo-Soundscapes-Audio/600_Sounds/human/
  inflating: Emo-Soundscapes/Emo-Soundscapes-Audio/600_Sounds/human/r_0human101979_1313063-hq.wav  
   creating: __MACOSX/Emo-Soundscapes/Emo-Soundscapes-Audio/600_So

In [0]:
# import models and utils ...
%tensorflow_version 1.x
# needed for compatibility ...
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()

from samplernn import AudioReader
from samplernn import SampleRnnModel
from samplernn import mu_law_decode
from samplernn import optimizer_factory
from train import *

In [0]:
# set some hyperparameters ... 
# trying to learn nature sounds ..
DATA_DIRECTORY = './corpus/Emo-Soundscapes/Emo-Soundscapes-Audio/600_Sounds/nature'
LOGDIR_ROOT = './logdir'
CHECKPOINT_EVERY = 5
GENERATE_EVERY = 8
NUM_STEPS = 2000
LEARNING_RATE = 1e-3
SAMPLE_SIZE = 100000
L2_REGULARIZATION_STRENGTH = 0
SILENCE_THRESHOLD = None
MOMENTUM = 0.9
MAX_TO_KEEP = 5

N_SECS = 3
SAMPLE_RATE = 22050
LENGTH = N_SECS * SAMPLE_RATE

BATCH_SIZE = 1
NUM_GPU = 1

BIG_FRAME_SIZE = 8
FRAME_SIZE = 2
Q_LEVELS = 256
RNN_TYPE = 'GRU'
DIM = 1024
N_RNN = 1
SEQ_LEN = 520
EMB_SIZE = 256
OPTIMIZER = 'adam'

In [11]:
# prepare stuff
logdir = os.path.join(LOGDIR_ROOT, 'train')
coord = tf.train.Coordinator()

# prepare data
with tf.name_scope('create_inputs'):
    reader = AudioReader(DATA_DIRECTORY,
                         coord,
                         sample_rate = SAMPLE_RATE,
                         sample_size = SAMPLE_SIZE,
                         silence_threshold = SILENCE_THRESHOLD)
    audio_batch = reader.dequeue(BATCH_SIZE)

In [12]:
# create the model
net = SampleRnnModel(batch_size = BATCH_SIZE,
                     big_frame_size = BIG_FRAME_SIZE,
                     frame_size = FRAME_SIZE,
                     q_levels = Q_LEVELS,
                     rnn_type = RNN_TYPE,
                     dim = DIM,
                     n_rnn = N_RNN,
                     seq_len = SEQ_LEN,
                     emb_size = EMB_SIZE)

optim = optimizer_factory[OPTIMIZER](learning_rate=LEARNING_RATE, momentum=MOMENTUM)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.



In [0]:
# get the global step variable
global_step = tf.get_variable('global_step',
                              [],
                              initializer=tf.constant_initializer(0),
                              trainable=False)

In [0]:
######Multi GPU###########
tower_grads = []
losses = []
train_input_batch_rnn = []
train_big_frame_state = []
train_frame_state = []
final_big_frame_state = []
final_frame_state = []
for i in range(NUM_GPU):
    train_input_batch_rnn.append(
        tf.Variable(
            tf.zeros(
                [net.batch_size, net.seq_len, 1]
            ),
            trainable=False,
            name="input_batch_rnn",
            dtype=tf.float32
        )
    )
    train_big_frame_state.append(
        net.big_cell.zero_state(net.batch_size, tf.float32))
    final_big_frame_state.append(
        net.big_cell.zero_state(net.batch_size, tf.float32))
    train_frame_state.append(
        net.cell.zero_state(net.batch_size, tf.float32))
    final_frame_state.append(
        net.cell.zero_state(net.batch_size, tf.float32))
    
with tf.variable_scope(tf.get_variable_scope()):
    for i in range(NUM_GPU):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('TOWER_%d' % (i)) as scope:
                # Create model.
                print("Creating model On Gpu:%d." % (i))
                (
                    loss,
                    final_big_frame_state[i],
                    final_frame_state[i]
                )=net.loss_SampleRnn(
                    train_input_batch_rnn[i],
                    train_big_frame_state[i],
                    train_frame_state[i],
                    l2_regularization_strength=None  # noqa: E501
                )              
                tf.get_variable_scope().reuse_variables()
                losses.append(loss)
                # Reuse variables for the next tower.
                trainable = tf.trainable_variables()
                gradients = optim.compute_gradients(
                    loss,
                    trainable,
                    aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N   # noqa: E501
                )
                tower_grads.append(gradients)
grad_vars = average_gradients(tower_grads)
grads, vars = list(zip(*grad_vars))
grads_clipped, _ = tf.clip_by_global_norm(grads, 5.0)
grad_vars = list(zip(grads_clipped, vars))

for name in grad_vars:
    print(name)

apply_gradient_op = optim.apply_gradients(
    grad_vars, global_step=global_step)
#################
infe_para = create_gen_wav_para(net)

writer = tf.summary.FileWriter(logdir)
writer.add_graph(tf.get_default_graph())
#run_metadata = tf.RunMetadata()
summaries = tf.summary.merge_all()

tf_config = tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=False)
tf_config.gpu_options.allow_growth = True
sess = tf.Session(config=tf_config)

init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver(var_list=tf.trainable_variables(),
                       max_to_keep=MAX_TO_KEEP)

try:
    saved_global_step = load(saver, sess, logdir)
    if saved_global_step is None:
        # The first training step will be saved_global_step + 1,
        # therefore we put -1 here for new or overwritten trainings.
        saved_global_step = -1

except:
    print("Something went wrong while restoring checkpoint. "
          "We will terminate training to avoid accidentally overwriting "
          "the previous model.")
    raise

threads = tf.train.start_queue_runners(sess=sess, coord=coord)
reader.start_threads(sess)

step = None
last_saved_step = saved_global_step
try:
    for step in range(saved_global_step + 1, NUM_STEPS):
        if (step-1) % GENERATE_EVERY == 0 and step > GENERATE_EVERY:
            generate_and_save_samples(step, net, infe_para, sess)

        final_big_s = []
        final_s = []
        for g in range(NUM_GPU):
            final_big_s.append(sess.run(net.big_initial_state))
            final_s.append(sess.run(net.initial_state))
        start_time = time.time()
        inputslist = [sess.run(audio_batch) for i in range(NUM_GPU)]
        loss_sum = 0
        idx_begin = 0
        audio_length = SAMPLE_SIZE - BIG_FRAME_SIZE
        bptt_length = SEQ_LEN - BIG_FRAME_SIZE
        stateful_rnn_length = audio_length // bptt_length
        outp_list = [summaries,
                     losses,
                     apply_gradient_op,
                     final_big_frame_state,
                     final_frame_state]
        for i in range(0, stateful_rnn_length):
            inp_dict = {}
            for g in range(NUM_GPU):
                inp_dict[train_input_batch_rnn[g]] = \
                    inputslist[g][:, idx_begin: idx_begin+SEQ_LEN, :]
                inp_dict[train_big_frame_state[g]] = final_big_s[g]
                inp_dict[train_frame_state[g]] = final_s[g]
            idx_begin += SEQ_LEN-BIG_FRAME_SIZE

            summary, loss_gpus, _, final_big_s, final_s = \
                sess.run(outp_list, feed_dict=inp_dict)
            writer.add_summary(summary, step)
            for g in range(NUM_GPU):
                loss_gpu = loss_gpus[g]/stateful_rnn_length
                loss_sum += loss_gpu/NUM_GPU
        duration = time.time() - start_time
        print('step {:d} - loss = {:.3f}, ({:.3f} sec/step)'
              .format(step, loss_sum, duration))

        if step % CHECKPOINT_EVERY == 0:
            save(saver, sess, logdir, step)
            last_saved_step = step

except KeyboardInterrupt:
    # Introduce a line break after ^C is displayed so save message
    # is on its own line.
    print()
finally:
    if step > last_saved_step:
        save(saver, sess, logdir, step)
    coord.request_stop()
    coord.join(threads)

Creating model On Gpu:0.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
(<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_0:0' shape=(1024, 4096) dtype=float32>, <tf.Variable 'BigFrame_layer/big_frame_proj_weights:0' shape=(1024, 4096) dtype=float32_ref>)
(<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(1032, 2048) dtype=float32>, <tf.Variable 'BigFrame_layer/BIG_FRAME_RNN/gru_cell/gates/kernel:0' shape=(1032, 2048) dtype=floa